# Isochrone Generator from Point Layers

This notebook generates isochrone buffers around point layers using OpenRouteService API.

### Import Libraries

In [2]:
import geopandas as gpd
import pandas as pd

import os
import time

from pathlib import Path

import openrouteservice as ors
from openrouteservice.isochrone import isochrone

import _params as params

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'openrouteservice.isochrone'

### Configuration

In [ ]:
# Set API key directly (run once, then comment out)
# os.environ['ORS_API_KEY'] = 'your_api_key_here'

# Set your OpenRouteService API key
client = ors.Client(key=os.getenv('ORS_API_KEY'))

In [ ]:
# Create output folder
isochrone_dir = Path('./isochrones')
isochrone_dir.mkdir(exist_ok=True)

## Walking Isochrones

### Load Point data for Walking Isochrones

In [ ]:
# List of your point layers
points_walk = [
    params.strTransitIn_Stops,    # Transit stops
    params.strSchoolsIn,          # Schools
    params.strChildCareIn,        # Childcare
    params.strCommunityCenterIn   # Community centers
]

layers_walk = {}

for file_path in points_walk:
    try:
        name = os.path.splitext(os.path.basename(file_path))[0]
        gdf = gpd.read_file(file_path)

        if not all(gdf.geometry.geom_type == 'Point'):
            continue

        if gdf.crs is None:
            gdf = gdf.set_crs('EPSG:4326')
        elif gdf.crs.to_epsg() != 4326:
            gdf = gdf.to_crs('EPSG:4326')

        layers_walk[name] = gdf
        print(f"Loaded {name}: {len(gdf)} points")

    except Exception as e:
        print(f"Error {name}: {e}")

### Generate Walking Distance Isochrones

In [ ]:
# Parameters
travel_mode = 'foot-walking'
ranges = 600  # in seconds; 10 minutes

# Loop through each walking layer
for layer_name, gdf in layers_walk.items():
    print(f"Processing {layer_name}...")

    # Generate isochrones
    all_isochrones = []

    for i, row in gdf.iterrows():
        coords = [row.geometry.x, row.geometry.y]

        try:
            response = isochrone(
                client=client,
                locations=[coords],
                profile=travel_mode,
                range_type='time',
                range=ranges
            )

            iso_gdf = gpd.GeoDataFrame.from_features(response['features'], crs='EPSG:4326')
            iso_gdf['point_id'] = row[gdf.columns[0]]
            all_isochrones.append(iso_gdf)

        except Exception as e:
            print(f"Failed: {e}")

        time.sleep(2)

    # Save layer
    if all_isochrones:
        result = pd.concat(all_isochrones, ignore_index=True)
        filename = f"{layer_name}_iso_{travel_mode}_{ranges//60}min.geojson"
        result.to_file(f"./isochrones/{filename}")
        print(f"Saved {filename}")

## Driving Isochrones

### Load Point data for Driving Isochrones

In [ ]:
# Driving layers - regional destinations
points_drive = [
    params.strCentersIn,          # Regional centers
    params.strGroceryIn,          # Grocery stores
    params.strHealthCareIn        # Healthcare facilities
]

layers_drive = {}

for file_path in points_drive:
    try:
        name = os.path.splitext(os.path.basename(file_path))[0]
        gdf = gpd.read_file(file_path)

        if not all(gdf.geometry.geom_type == 'Point'):
            continue

        if gdf.crs is None:
            gdf = gdf.set_crs('EPSG:4326')
        elif gdf.crs.to_epsg() != 4326:
            gdf = gdf.to_crs('EPSG:4326')

        layers_drive[name] = gdf
        print(f"Loaded {name}: {len(gdf)} points")

    except Exception as e:
        print(f"Error {name}: {e}")

### Generate Driving Distance Isochrones

In [ ]:
# Parameters
travel_mode = 'driving-car'
ranges = 600  # in seconds; 10 minutes

# Loop through each driving layer
for layer_name, gdf in layers_drive.items():
    print(f"Processing {layer_name}...")

    # Generate isochrones
    all_isochrones = []

    for i, row in gdf.iterrows():
        coords = [row.geometry.x, row.geometry.y]

        try:
            response = isochrone(
                client=client,
                locations=[coords],
                profile=travel_mode,
                range_type='time',
                range=ranges
            )

            iso_gdf = gpd.GeoDataFrame.from_features(response['features'], crs='EPSG:4326')
            iso_gdf['point_id'] = row[gdf.columns[0]]
            all_isochrones.append(iso_gdf)

        except Exception as e:
            print(f"Failed: {e}")

        time.sleep(2)

    # Save layer
    if all_isochrones:
        result = pd.concat(all_isochrones, ignore_index=True)
        filename = f"{layer_name}_iso_{travel_mode}_{ranges//60}min.geojson"
        result.to_file(f"./isochrones/{filename}")
        print(f"Saved {filename}")

print("Complete!")